# How it works

1. User places CSV file from the Bank in the directory
2. Program imports it into a pandas dataframe cleans it up, fills in NaNs, extracts and adds the year and month

    The program works with 2 dictionaries - the Categoriser and the Category Mapper

    * The Categoriser is a list of key words that asssign a subcategory to each entry
    * The Category Mapper assigns categories and classes to subcategories


3. The program goes through the CSV line by line, scanning for subcategory key words. Three cases follow:

* Case 1: No results found --> the user is prompted to enter a new keyword/subcategory pair into the dictionary OR make it a once-off classification
* Case 2: One result found --> the program assigns the subcategory to the keyword
* Case 3: More than one result found --> the program is in danger of making a mistaken classification because the user hasn't chosen their keyword closely enough. The user is prompted to 


# Functions

In [2]:
import json
import os
import pandas as pd

In [3]:
%cd "F:/Google Drive/JupyterNotebooks/Useful Projects/Budget Tracker/"

# I operate across a couple of different computers. Sadly, they don't have the same path.
path = "F:/Google Drive/JupyterNotebooks/Useful Projects/Budget Tracker/"

F:\Google Drive\JupyterNotebooks\Useful Projects\Budget Tracker


## Display, Save and Load

In [4]:
def print_dict(my_dict):
    for key,value in sorted(my_dict.items()):
        print(key,":",value)

In [5]:
def load_dict_from_json(target_dict):
    with open("{}{}.json".format(path,target_dict),"r") as fh:
        return(json.load(fh))

In [6]:
def save_dict_to_json(filename,dict_to_save):
    with open("{}{}.json".format(path,filename),"w") as savefile:
        json.dump(dict_to_save,savefile,indent=2, sort_keys = True)

In [7]:
my_dict = {1:"one",2:"two",3:"three"}

save_dict_to_json("Test_Dict",my_dict)

## Dynamically adding things - helpful functions

In [8]:
def list_subcategory_options():
    category_map_dict = load_dict_from_json("Category_Map")
    subcats_list = sorted(category_map_dict.keys())
    subcat_chooser = dict(zip(range(len(subcats_list)),subcats_list))
    return(subcat_chooser)

In [9]:
list_subcategory_options()

{0: 'Adobe Subscriptions',
 1: 'Anniversary',
 2: 'Antivirus',
 3: 'Audible',
 4: 'Aussielent',
 5: 'Blinkist',
 6: 'Books',
 7: 'Boost',
 8: 'CARE Australia',
 9: 'Camera Gear',
 10: 'Cash',
 11: 'Centrelink',
 12: 'Clothing',
 13: 'Comprehensive Insurance',
 14: 'Computer',
 15: 'Courses',
 16: 'Credit Card Interest',
 17: 'Dad',
 18: 'Dan',
 19: 'Department of Education',
 20: 'Desexing/Medical',
 21: 'Dog Food',
 22: 'Dog Rego',
 23: 'Drone',
 24: 'Eating Out',
 25: 'Electricity',
 26: 'Exercise Equipment',
 27: 'Festivals',
 28: 'Food',
 29: 'Fuel',
 30: 'Gifts Received',
 31: 'Google Drive',
 32: 'Greenslip',
 33: 'Home Contents',
 34: 'Interest',
 35: 'Internet',
 36: 'Jim',
 37: 'Life Insurance',
 38: 'Mel Rent',
 39: 'Misc Other Income',
 40: 'Missy',
 41: 'Movies',
 42: 'Mum',
 43: 'Museums',
 44: 'NCTM',
 45: 'NRMA Membership',
 46: 'Naked Wines',
 47: 'Netflix/Stan',
 48: 'New Scientist',
 49: 'Other Gifts',
 50: 'Other Tax Deductible Purchases',
 51: 'Phone Upgrades',
 52:

In [10]:
def list_category_options():
    category_map_dict = load_dict_from_json("Category_Map")
    subcats_list = category_map_dict.keys()
    category_list = []
    for x in subcats_list:
        category_list.append(category_map_dict[x]["Category"])
    category_list = sorted(set(category_list)) 
    cat_chooser = dict(zip(range(len(category_list)),category_list))
    return(cat_chooser)

In [11]:
list_category_options()

{0: 'ATO',
 1: 'Bank',
 2: 'Car',
 3: 'Charities+Causes',
 4: 'Creativity+Learning',
 5: 'Debts+Loans',
 6: 'Department of Education',
 7: 'Dog',
 8: 'Entertainment',
 9: 'Gifts',
 10: 'Gifts Received',
 11: 'Life',
 12: 'Misc',
 13: 'Misc Other Income',
 14: 'Other Insurance',
 15: 'Rebate',
 16: 'Subscriptions',
 17: 'TFR',
 18: 'Tax Deductible Purchases',
 19: 'Technology',
 20: 'Test',
 21: 'Test2',
 22: 'Uncategorised',
 23: 'Utilities'}

In [46]:
def new_subcategory():
    category_map_dict = load_dict_from_json("Category_Map")
    
    #print out the pre-existing options
    print_dict(category_map_dict)
    
    new_subcat = input("What is your sub-category?")
    # testing to see if it already exists
    while new_subcat in category_map_dict:
# room for checking if this is in or NEARLY in the thing already    
        new_subcat = input("Oops. That category already extists. Choose another. ")
    # once we have a unique value, enter the category and class
    new_cat = input("What is your category?")
    new_class = input("What is your class?")
    
    #now actually adding the new category to the repository
    category_map_dict[new_subcat] = {"Category":new_cat,"Class":new_class}
    #save the file
    print("Adding new subcategory --> {}:{{{}:{}}}".format(new_subcat,new_cat,new_class))
    
    save_dict_to_json("Category_Map",category_map_dict)
    

In [47]:
new_subcategory()

Accreditation : {'Category': 'Misc School', 'Class': 'Expenses'}
Adobe Subscriptions : {'Category': 'Tax Deductible Purchases', 'Class': 'Expenses'}
Anniversary : {'Category': 'Gifts', 'Class': 'Expenses'}
Antivirus : {'Category': 'Technology', 'Class': 'Expenses'}
Audible : {'Category': 'Subscriptions', 'Class': 'Expenses'}
Aussielent : {'Category': 'Life', 'Class': 'Expenses'}
Blinkist : {'Category': 'Subscriptions', 'Class': 'Expenses'}
Books : {'Category': 'Creativity+Learning', 'Class': 'Expenses'}
Boost : {'Category': 'Utilities', 'Class': 'Expenses'}
CARE Australia : {'Category': 'Charities+Causes', 'Class': 'Expenses'}
Camera Gear : {'Category': 'Technology', 'Class': 'Expenses'}
Cash : {'Category': 'Misc', 'Class': 'Expenses'}
Centrelink : {'Category': 'Debts+Loans', 'Class': 'Expenses'}
Clothing : {'Category': 'Life', 'Class': 'Expenses'}
Comprehensive Insurance : {'Category': 'Car', 'Class': 'Expenses'}
Computer : {'Category': 'Technology', 'Class': 'Expenses'}
Courses : {'C

In [40]:
new_subcategory()
# passed test

Adobe Subscriptions : {'Category': 'Tax Deductible Purchases', 'Class': 'Expenses'}
Anniversary : {'Category': 'Gifts', 'Class': 'Expenses'}
Antivirus : {'Category': 'Technology', 'Class': 'Expenses'}
Audible : {'Category': 'Subscriptions', 'Class': 'Expenses'}
Aussielent : {'Category': 'Life', 'Class': 'Expenses'}
Blinkist : {'Category': 'Subscriptions', 'Class': 'Expenses'}
Books : {'Category': 'Creativity+Learning', 'Class': 'Expenses'}
Boost : {'Category': 'Utilities', 'Class': 'Expenses'}
CARE Australia : {'Category': 'Charities+Causes', 'Class': 'Expenses'}
Camera Gear : {'Category': 'Technology', 'Class': 'Expenses'}
Cash : {'Category': 'Misc', 'Class': 'Expenses'}
Centrelink : {'Category': 'Debts+Loans', 'Class': 'Expenses'}
Clothing : {'Category': 'Life', 'Class': 'Expenses'}
Comprehensive Insurance : {'Category': 'Car', 'Class': 'Expenses'}
Computer : {'Category': 'Technology', 'Class': 'Expenses'}
Courses : {'Category': 'Tax Deductible Purchases', 'Class': 'Expenses'}
Credit

In [14]:
def kw_match_tracker(eachDescription):
    with open("{}AutoCatV2.json".format(path),"r") as keyword_map:
        keyword_map = json.load(keyword_map)
        
    keyword_list = list(keyword_map.keys())
    hit_count = 0
    
    match_dict = {}
    for eachKey in keyword_list:
        #print("searching for {} in {}".format(eachKey,eachDescription))
        if eachKey in eachDescription:
            hit_count += 1
            #print("Hit count: ",str(hit_count))
            match_dict[hit_count] = eachKey
    
    output = {"match_dict":match_dict,"hit_count":hit_count}
    return(output)

In [15]:
desc = "TFR to 224238S6 ONLINE To-P W NEWMAN Ref-Savings Savings"
kw_match_tracker(desc)

{'hit_count': 2, 'match_dict': {1: '224238S6', 2: 'TFR'}}

In [37]:
def new_keyword():
    keyword_map = load_dict_from_json("AutoCatV2")
    
    new_kw = input("Please choose a keyword: ")
    print("New keyword: {} ".format(new_kw))
    options = list_subcategory_options()
    
    print_dict(options)
    
    new_choice = str.upper(input("To enter a new subcategory, type N. Type anything else to use the current options"))
    if new_choice == "N":
        new_subcategory()
        # refresh list
        options = list_subcategory_options()
        print_dict(options)
    
    subcat_choice = int(input("Enter the number of the subcategory mapping you want to use: "))
    
    subcat = options[subcat_choice]
    keyword_map[new_kw]  = subcat
    
    save_option = str.upper(input("Save new keyword mapping {} : {}? Press Y ".format(new_kw,subcat)))
    
    if save_option == "Y":
        save_dict_to_json("AutoCatV2",keyword_map)
    
    return(subcat)

In [17]:
#new_keyword()
#passed test

## Pandas Functions

In [18]:
# to be called in an apply in pandas
# goes through the df reading dates and adds the month
# dependency: dates column

def pd_add_month(eachDate):
    return(eachDate.month)

In [19]:
# to be called in an apply in pandas
# goes through the df reading dates and adds the month
# dependency: dates column

def pd_add_year(eachDate):
    return(eachDate.year)

In [20]:
# to be called in an apply in pandas
# goes through the df reading descriptions and adds the subcategory using a keyword search

def pd_add_subcategories(eachDescription):
    
    match_dict = kw_match_tracker(eachDescription)["match_dict"]
    hit_count = kw_match_tracker(eachDescription)["hit_count"]                                                            #"{} --> {}".format(eachKey,keyword_map[eachKey])
    keyword_map = load_dict_from_json("AutoCatV2")
    
    if hit_count == 0:
        print("No Matches found for '",eachDescription,"'")
        user_choice = str.upper(input("Type N for new keyword, C to categorise as a once-off or S for skip"))
        if user_choice == "N":
            return(new_keyword())
            
        elif user_choice == "C":
            print_dict(list_subcategory_options())
            subcat = int(input("Make your choice from the list above by entering the number"))
            return(list_subcategory_options()[subcat])
        
        elif user_choice == "S":
            return("Uncategorised")
    
    elif hit_count == 1:
        return(keyword_map[match_dict[hit_count]])
    
    elif hit_count > 1:
        print("More than one hit found for '{}'".format(eachDescription))
        print_dict(match_dict)
        shortlist_choice = int(input("Please choose a number from the options above or anything else to skip."))
        if shortlist_choice in range(hit_count):
            return(keyword_map[match_dict[shortlist_choice]])
        else:
            return("Uncategorised")

In [21]:
# to be called in an apply in pandas
# goes through the df reading subcategories and adds the category
# dependency: subcategory column

def pd_add_categories(eachSubcategory):
    
    category_map_dict = load_dict_from_json("Category_Map")
    
    if eachSubcategory in category_map_dict: #this line is probably not necessary as you have to subcategorise first...
        return(category_map_dict[eachSubcategory]["Category"])
    
    else:
        return("Uncategorised")

In [22]:
test = "Supplies"
pd_add_categories(test)

'Life'

In [23]:
# to be called in an apply in pandas
# goes through the df reading subcategories and adds the class
# dependency: subcategory column

def pd_add_classes(eachSubcategory):
    
    category_map_dict = load_dict_from_json("Category_Map")
    
    if eachSubcategory in category_map_dict:
        return(category_map_dict[eachSubcategory]["Class"])
    
    else:
        return("Uncategorised")

In [24]:
test = "Supplies"
pd_add_classes(test)

'Expenses'

## Actions for the main loop

In [25]:
def pd_import_data(filename):
    return(pd.read_excel("{}{}.xlsx".format(path,filename)))

In [26]:
df = pd_import_data("March April 2015")
df.head(5)

,Bank Account,Date,Backdate,Description,Transaction,Balance
0,S1,2015-03-01,2015-02-28,TFR TO 484799 502175908 ONLINE To-MISS MELISSA...,-200.00,310.95
1,S1,2015-03-01,NaT,TFR TO 032001 143827 ONLINE To-MUNGINDI CENTRA...,-105.00,205.95
2,S1,2015-03-02,NaT,TELSTRAKENAN20EASYPAYA From: Telstra DDebit Re...,-147.97,57.98
3,S1,2015-03-11,NaT,DIRECT CREDIT From: DEPT OF SCHOOL E Ref: DOSE...,1827.70,1885.68
4,S1,2015-03-12,NaT,TF From: TF Ref: NSW Teachers Feder,-29.95,1855.73


In [27]:
#cleans data

def clean_data(df):
    df.fillna("NA",inplace = True)

In [28]:
def add_year(df):
    df["Year"] = df.apply(lambda x: pd_add_year(x["Date"]),axis= 1)

In [29]:
df = pd.read_excel("March April 2015.xlsx")
clean_data(df)
add_year(df)
df.head()

,Bank Account,Date,Backdate,Description,Transaction,Balance,Year
0,S1,2015-03-01,2015-02-28 00:00:00,TFR TO 484799 502175908 ONLINE To-MISS MELISSA...,-200.00,310.95,2015
1,S1,2015-03-01,NA,TFR TO 032001 143827 ONLINE To-MUNGINDI CENTRA...,-105.00,205.95,2015
2,S1,2015-03-02,NA,TELSTRAKENAN20EASYPAYA From: Telstra DDebit Re...,-147.97,57.98,2015
3,S1,2015-03-11,NA,DIRECT CREDIT From: DEPT OF SCHOOL E Ref: DOSE...,1827.70,1885.68,2015
4,S1,2015-03-12,NA,TF From: TF Ref: NSW Teachers Feder,-29.95,1855.73,2015


In [30]:
def add_month(df):
    df["Month"] = df.apply(lambda x: pd_add_month(x["Date"]),axis= 1)

In [31]:
df = pd.read_excel("March April 2015.xlsx")
clean_data(df)
add_month(df)
df.head()

,Bank Account,Date,Backdate,Description,Transaction,Balance,Month
0,S1,2015-03-01,2015-02-28 00:00:00,TFR TO 484799 502175908 ONLINE To-MISS MELISSA...,-200.00,310.95,3
1,S1,2015-03-01,NA,TFR TO 032001 143827 ONLINE To-MUNGINDI CENTRA...,-105.00,205.95,3
2,S1,2015-03-02,NA,TELSTRAKENAN20EASYPAYA From: Telstra DDebit Re...,-147.97,57.98,3
3,S1,2015-03-11,NA,DIRECT CREDIT From: DEPT OF SCHOOL E Ref: DOSE...,1827.70,1885.68,3
4,S1,2015-03-12,NA,TF From: TF Ref: NSW Teachers Feder,-29.95,1855.73,3


In [33]:
def add_subcategories(df):
    df["Subcategory"] = df.apply(lambda x: pd_add_subcategories(x["Description"]),axis=1)

In [35]:
df = pd.read_excel("March April 2015.xlsx")
clean_data(df)
add_subcategories(df)
df

No Matches found for ' TFR TO 484799 502175908 ONLINE To-MISS MELISSA KATHRYN JONES Ref-Some spending mula To Mel '
Type N for new keyword, C to categorise as a once-off or S for skipN
Please choose a keyword: MELISSA KATHRYN JONES
New keyword: MELISSA KATHRYN JONES 
0 : Adobe Subscriptions
1 : Anniversary
2 : Antivirus
3 : Audible
4 : Aussielent
5 : Blinkist
6 : Books
7 : Boost
8 : CARE Australia
9 : Camera Gear
10 : Cash
11 : Centrelink
12 : Clothing
13 : Comprehensive Insurance
14 : Computer
15 : Courses
16 : Credit Card Interest
17 : Dad
18 : Dan
19 : Department of Education
20 : Desexing/Medical
21 : Dog Food
22 : Dog Rego
23 : Drone
24 : Eating Out
25 : Electricity
26 : Exercise Equipment
27 : Festivals
28 : Food
29 : Fuel
30 : Gifts Received
31 : Google Drive
32 : Greenslip
33 : Home Contents
34 : Interest
35 : Internet
36 : Jim
37 : Life Insurance
38 : Mel Rent
39 : Misc Other Income
40 : Missy
41 : Movies
42 : Mum
43 : Museums
44 : NCTM
45 : NRMA Membership
46 : Naked Wines
47

,Bank Account,Date,Backdate,Description,Transaction,Balance,Subcategory
0,S1,2015-03-01,2015-02-28 00:00:00,TFR TO 484799 502175908 ONLINE To-MISS MELISSA...,-200.00,310.95,Missy
1,S1,2015-03-01,NA,TFR TO 032001 143827 ONLINE To-MUNGINDI CENTRA...,-105.00,205.95,Clothing
2,S1,2015-03-02,NA,TELSTRAKENAN20EASYPAYA From: Telstra DDebit Re...,-147.97,57.98,Internet
3,S1,2015-03-11,NA,DIRECT CREDIT From: DEPT OF SCHOOL E Ref: DOSE...,1827.70,1885.68,Department of Education
4,S1,2015-03-12,NA,TF From: TF Ref: NSW Teachers Feder,-29.95,1855.73,Teacher's Fed
5,S1,2015-03-15,NA,TFR TO 932000 652932 ONLINE To-MR PETER NEWMAN...,-780.00,1075.73,TFR
6,S1,2015-03-15,NA,TFR to 224238S6 ONLINE To-P W NEWMAN Ref-Savin...,-700.00,375.73,TFR
7,S1,2015-03-22,NA,TFR TO 932000 652932 MOB To-MR PETER NEWMAN Re...,-200.00,175.73,TFR
8,S1,2015-03-25,NA,DIRECT CREDIT From: DEPT OF SCHOOL E Ref: DOSE...,1827.70,2003.43,Department of Education
9,S1,2015-03-25,NA,TFR TO 484799 502175908 ONLINE To-MISS MELISSA...,-575.00,1428.43,Missy


In [52]:
# dependency: add_subcategories
def add_categories(df):
    df["Category"] = df.apply(lambda x: pd_add_categories(x["Subcategory"]),axis=1)

In [53]:
df = pd.read_excel("March April 2015.xlsx")
clean_data(df)
add_subcategories(df)
add_categories(df)
df

No Matches found for ' PAYPAL AUSTRALIA From: PAYPAL AUSTRALIA Ref: 5D8PPWSZ9VZEC '
Type N for new keyword, C to categorise as a once-off or S for skipS
No Matches found for ' PAYPAL AUSTRALIA From: PAYPAL AUSTRALIA Ref: 5D8VDTKHR6ZEC '
Type N for new keyword, C to categorise as a once-off or S for skipS
No Matches found for ' PAYPAL AUSTRALIA From: PAYPAL AUSTRALIA Ref: J222224NMVUG6 '
Type N for new keyword, C to categorise as a once-off or S for skipS
No Matches found for ' PAYPAL AUSTRALIA From: PAYPAL AUSTRALIA Ref: J222224NTMKBN '
Type N for new keyword, C to categorise as a once-off or S for skipS


,Bank Account,Date,Backdate,Description,Transaction,Balance,Subcategory,Category
0,S1,2015-03-01,2015-02-28 00:00:00,TFR TO 484799 502175908 ONLINE To-MISS MELISSA...,-200.00,310.95,Missy,Gifts
1,S1,2015-03-01,NA,TFR TO 032001 143827 ONLINE To-MUNGINDI CENTRA...,-105.00,205.95,Clothing,Life
2,S1,2015-03-02,NA,TELSTRAKENAN20EASYPAYA From: Telstra DDebit Re...,-147.97,57.98,Internet,Utilities
3,S1,2015-03-11,NA,DIRECT CREDIT From: DEPT OF SCHOOL E Ref: DOSE...,1827.70,1885.68,Department of Education,Department of Education
4,S1,2015-03-12,NA,TF From: TF Ref: NSW Teachers Feder,-29.95,1855.73,Teacher's Fed,Tax Deductible Purchases
5,S1,2015-03-15,NA,TFR TO 932000 652932 ONLINE To-MR PETER NEWMAN...,-780.00,1075.73,TFR,TFR
6,S1,2015-03-15,NA,TFR to 224238S6 ONLINE To-P W NEWMAN Ref-Savin...,-700.00,375.73,TFR,TFR
7,S1,2015-03-22,NA,TFR TO 932000 652932 MOB To-MR PETER NEWMAN Re...,-200.00,175.73,TFR,TFR
8,S1,2015-03-25,NA,DIRECT CREDIT From: DEPT OF SCHOOL E Ref: DOSE...,1827.70,2003.43,Department of Education,Department of Education
9,S1,2015-03-25,NA,TFR TO 484799 502175908 ONLINE To-MISS MELISSA...,-575.00,1428.43,Missy,Gifts


In [54]:
# dependency: add_subcategories
def add_classes(df):
    df["Class"] = df.apply(lambda x: pd_add_classes(x["Subcategory"]),axis=1)

In [55]:
df = pd.read_excel("March April 2015.xlsx")
clean_data(df)
add_subcategories(df)
add_classes(df)
df

No Matches found for ' PAYPAL AUSTRALIA From: PAYPAL AUSTRALIA Ref: 5D8PPWSZ9VZEC '
Type N for new keyword, C to categorise as a once-off or S for skipS
No Matches found for ' PAYPAL AUSTRALIA From: PAYPAL AUSTRALIA Ref: 5D8VDTKHR6ZEC '
Type N for new keyword, C to categorise as a once-off or S for skipS
No Matches found for ' PAYPAL AUSTRALIA From: PAYPAL AUSTRALIA Ref: J222224NMVUG6 '
Type N for new keyword, C to categorise as a once-off or S for skipS
No Matches found for ' PAYPAL AUSTRALIA From: PAYPAL AUSTRALIA Ref: J222224NTMKBN '
Type N for new keyword, C to categorise as a once-off or S for skipS


,Bank Account,Date,Backdate,Description,Transaction,Balance,Subcategory,Class
0,S1,2015-03-01,2015-02-28 00:00:00,TFR TO 484799 502175908 ONLINE To-MISS MELISSA...,-200.00,310.95,Missy,Expenses
1,S1,2015-03-01,NA,TFR TO 032001 143827 ONLINE To-MUNGINDI CENTRA...,-105.00,205.95,Clothing,Expenses
2,S1,2015-03-02,NA,TELSTRAKENAN20EASYPAYA From: Telstra DDebit Re...,-147.97,57.98,Internet,Expenses
3,S1,2015-03-11,NA,DIRECT CREDIT From: DEPT OF SCHOOL E Ref: DOSE...,1827.70,1885.68,Department of Education,Income
4,S1,2015-03-12,NA,TF From: TF Ref: NSW Teachers Feder,-29.95,1855.73,Teacher's Fed,Expenses
5,S1,2015-03-15,NA,TFR TO 932000 652932 ONLINE To-MR PETER NEWMAN...,-780.00,1075.73,TFR,TFR
6,S1,2015-03-15,NA,TFR to 224238S6 ONLINE To-P W NEWMAN Ref-Savin...,-700.00,375.73,TFR,TFR
7,S1,2015-03-22,NA,TFR TO 932000 652932 MOB To-MR PETER NEWMAN Re...,-200.00,175.73,TFR,TFR
8,S1,2015-03-25,NA,DIRECT CREDIT From: DEPT OF SCHOOL E Ref: DOSE...,1827.70,2003.43,Department of Education,Income
9,S1,2015-03-25,NA,TFR TO 484799 502175908 ONLINE To-MISS MELISSA...,-575.00,1428.43,Missy,Expenses


In [48]:
def save_to_excel(df,filename,sheetname):
    path_name = "{}{}.xlsx".format(path,filename)
    print("Saving to",path_name+"?")
    if str.upper(input("Continue? Y or N")) == "Y":
# insert check for overwrite of file here
        df.to_excel(path_name,sheet_name=sheetname)
    elif "N":
        print("Save cancelled")

In [56]:
df = pd.read_excel("March April 2015.xlsx")
clean_data(df)
add_subcategories(df)
df

No Matches found for ' PAYPAL AUSTRALIA From: PAYPAL AUSTRALIA Ref: 5D8PPWSZ9VZEC '
Type N for new keyword, C to categorise as a once-off or S for skipS
No Matches found for ' PAYPAL AUSTRALIA From: PAYPAL AUSTRALIA Ref: 5D8VDTKHR6ZEC '
Type N for new keyword, C to categorise as a once-off or S for skipS
No Matches found for ' PAYPAL AUSTRALIA From: PAYPAL AUSTRALIA Ref: J222224NMVUG6 '
Type N for new keyword, C to categorise as a once-off or S for skips
No Matches found for ' PAYPAL AUSTRALIA From: PAYPAL AUSTRALIA Ref: J222224NTMKBN '
Type N for new keyword, C to categorise as a once-off or S for skips


,Bank Account,Date,Backdate,Description,Transaction,Balance,Subcategory
0,S1,2015-03-01,2015-02-28 00:00:00,TFR TO 484799 502175908 ONLINE To-MISS MELISSA...,-200.00,310.95,Missy
1,S1,2015-03-01,NA,TFR TO 032001 143827 ONLINE To-MUNGINDI CENTRA...,-105.00,205.95,Clothing
2,S1,2015-03-02,NA,TELSTRAKENAN20EASYPAYA From: Telstra DDebit Re...,-147.97,57.98,Internet
3,S1,2015-03-11,NA,DIRECT CREDIT From: DEPT OF SCHOOL E Ref: DOSE...,1827.70,1885.68,Department of Education
4,S1,2015-03-12,NA,TF From: TF Ref: NSW Teachers Feder,-29.95,1855.73,Teacher's Fed
5,S1,2015-03-15,NA,TFR TO 932000 652932 ONLINE To-MR PETER NEWMAN...,-780.00,1075.73,TFR
6,S1,2015-03-15,NA,TFR to 224238S6 ONLINE To-P W NEWMAN Ref-Savin...,-700.00,375.73,TFR
7,S1,2015-03-22,NA,TFR TO 932000 652932 MOB To-MR PETER NEWMAN Re...,-200.00,175.73,TFR
8,S1,2015-03-25,NA,DIRECT CREDIT From: DEPT OF SCHOOL E Ref: DOSE...,1827.70,2003.43,Department of Education
9,S1,2015-03-25,NA,TFR TO 484799 502175908 ONLINE To-MISS MELISSA...,-575.00,1428.43,Missy


In [57]:
save_to_excel(df,"Export 1","March-April 2015")

Saving to F:/Google Drive/JupyterNotebooks/Useful Projects/Budget Tracker/Export 1.xlsx?
Continue? Y or NY


## Main

In [ ]:
def main(filename):
    df = pd_import_data(filename)
    clean_data(df)
    add_year(df)
    add_month(df)
    add_subcategories(df)
    add_categories(df)
    add_classes(df)
    save_filename = input("Exporting to Excel. Filename: ")
    save_excel(df,"{}{}".format(filename,)

# Testing Area

In [ ]:
os.listdir()

In [51]:
df= pd.read_excel("March April 2015.xlsx")
clean_data(df)
#df.fillna("NA",inplace=True)
df["Subcategory"] = df.apply(lambda x: assign_subcategory(df["Description"]),axis= 1)

No Matches found for ' 0     TFR TO 484799 502175908 ONLINE To-MISS MELISSA...
1     TFR TO 032001 143827 ONLINE To-MUNGINDI CENTRA...
2     TELSTRAKENAN20EASYPAYA From: Telstra DDebit Re...
3     DIRECT CREDIT From: DEPT OF SCHOOL E Ref: DOSE...
4                   TF From: TF Ref: NSW Teachers Feder
5     TFR TO 932000 652932 ONLINE To-MR PETER NEWMAN...
6     TFR to 224238S6 ONLINE To-P W NEWMAN Ref-Savin...
7     TFR TO 932000 652932 MOB To-MR PETER NEWMAN Re...
8     DIRECT CREDIT From: DEPT OF SCHOOL E Ref: DOSE...
9     TFR TO 484799 502175908 ONLINE To-MISS MELISSA...
10    TFR to 224238S6 ONLINE To-P W NEWMAN Ref-Savin...
11                  TF From: TF Ref: NSW Teachers Feder
12    DIRECT DEBIT From: TPG Internet Ref: DF3Q0PJTS...
13            BPAY BOSTES ONLINE Ref-2763787 #134329061
14    TELSTRAKENAN20EASYPAYA From: Telstra DDebit Re...
15    DIRECT CREDIT From: DEPT OF SCHOOL E Ref: DOSE...
16                  TF From: TF Ref: NSW Teachers Feder
17    TFR TO 932000 65293

NameError: ("name 'new_keyword' is not defined", 'occurred at index 1')